In [16]:
# Initial setup
# Set up a dictionary of signs
signs = dict()
signs['union'] = 'v'
signs['intersection'] = '^'
signs['implication'] = '→'
signs['negation'] = '~'

# Set up list of atomic statements
atomic = []
atomic.append('A')
atomic.append('B')
atomic.append('C')
atomic.append('D')
atomic.append('E')

In [17]:
# To-do: Komla
# Function to transform parenthesis form (a->b)->(~b->~a) to non-parenthesis form ->->ab->~b~a

In [18]:
# Function to split text into Alpha, Beta, S1 and S2
def alpha_beta_split(text):

    # Support for switching between alpha to beta
    alpha_count = 0
    label = 'S1'

    # Dictionary of alpha/beta/S strings
    ab_dict = dict()
    ab_dict['alpha'] = ''
    ab_dict['beta'] = ''
    ab_dict['S1'] = ''
    ab_dict['S2'] = ''

    for i in range(len(text)):  # Loop over every char
        # Switch labels to beta or S2 if conditions are satisfied
        if alpha_count >= 1 and label == 'alpha':
            label = 'beta'
        elif text[i] == ',' and label == 'beta':
            label = 'S2'

        # If current char is a sign
        if text[i] in signs.values():
            if text[i] == signs['negation']:    # Ignore alpha_count for negation
                ab_dict[label] += text[i]       # Add negation to alpha/beta/S
            else:
                if label == 'S1':               # Marks start of alpha
                    label = 'alpha'
                else:
                    ab_dict[label] += text[i]       # Add sign to alpha/beta/S
                    alpha_count -= 1                # Reduce alpha_count to add 1 more atomic statement to alpha

        # If current char is an Atomic statement
        elif text[i] in atomic:
            if label != 'alpha':        # Ignore alpha_count for other labels
                ab_dict[label] += text[i]
            else:
                ab_dict[label] += text[i]
                alpha_count += 1

        # If current char is indecomposable sequence
        else:
            ab_dict[label] += text[i]

    return ab_dict

In [19]:
# Schema [S1,AvB,S2]/[S1,A,B,S2]
def union(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + ab_dict['alpha'] + ',' + ab_dict['beta'] + ab_dict['S2']

In [20]:
# Schema [S1,~(AvB),S2]/[S1,~A,S2;S1,~B,S2]
def neg_union(text):
    ab_dict = alpha_beta_split(text)
    return (ab_dict['S1'] + signs['negation'] + ab_dict['alpha'] + ab_dict['S2'], ab_dict['S1'] + signs['negation'] + ab_dict['beta'] + ab_dict['S2'])

In [21]:
# Schema [S1,(A^B),S2]/[S1,A,S2;S1,B,S2]
def intersection(text):
    ab_dict = alpha_beta_split(text)
    return (ab_dict['S1'] + ab_dict['alpha'] + ab_dict['S2'], ab_dict['S1'] + ab_dict['beta'] + ab_dict['S2'])

In [22]:
# Schema [S1,~~A,S2]/[S1,A,S2]
def double_negation(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + ab_dict['alpha'] + ab_dict['S2']

In [23]:
# Schema [S1,~(A→B),S2]/[S1,A,S2;S1,~B,S2]
def neg_implication(text):
    ab_dict = alpha_beta_split(text)
    return (ab_dict['S1'] + ab_dict['alpha'] + ab_dict['S2'], ab_dict['S1'] + signs['negation'] + ab_dict['beta'] + ab_dict['S2'])

In [24]:
# Schema [S1,~(A^B),S2]/[S1,~A,~B,S2]
def neg_intersection(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + signs['negation'] + ab_dict['alpha'] + ',' + signs['negation'] + ab_dict['beta'] + ab_dict['S2']

In [25]:
# Schema [S1,A→B,S2]/[S1,~A,B,S2]
def implication(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + signs['negation'] + ab_dict['alpha'] + ',' + ab_dict['beta'] + ab_dict['S2']

In [26]:
# Assigns schema and returns output of that schema
def assign_schema(text):
    for i in range(len(text)):
        if text[i] in signs.values():               # Check for signs
            if text[i] == signs['negation']:        # Check if next values are signs and assign schema accordingly
                if text[i+1] == signs['intersection']:
                    text = text[:i] + text[i+1:]    # Remove negation sign
                    text = neg_intersection(text)
                    break
                elif text[i+1] == signs['union']:
                    text = text[:i] + text[i+1:]
                    text = neg_union(text)
                    break
                elif text[i+1] == signs['negation']:
                    text = text[:i] + text[i+2:]
                    text = double_negation(text)
                    break
                elif text[i+1] == signs['implication']:
                    text = text[:i] + text[i+1:]
                    text = neg_implication(text)
                    break

            else:                               # Assign schemas for non-negative functions
                if text[i] == signs['union']: text = union(text)
                elif text[i] == signs['intersection']: text = intersection(text)
                elif text[i] == signs['implication']: text = implication(text)
                break

    if not isinstance(text, tuple): text = (text,)     # Put string in a tuple
    return text

In [27]:
# Helper function to check if a leaf is fundamental
def check_fundamental(leaf_text):
    fundamental = False
    leaf_text = leaf_text.split(',')
    for i in range(len(leaf_text)):
        for j in range(i+1, len(leaf_text)):
            if leaf_text[i] == signs['negation'] + leaf_text[j] or leaf_text[j] == signs['negation'] + leaf_text[i]: fundamental = True
    return fundamental

In [28]:
# Perform depth-first search, output leaf values and if leaf is fundamental
leaf_nodes = []
fundamental = []
def RS_tree(text_input):
    for text_output in assign_schema(text_input):
        if len(fundamental) > 0:
            if not fundamental[-1]: break                          # Stop recursion if a leaf is not fundamental

        if text_input != text_output: RS_tree(text_output)     # Check if leaf has been reached
        else:
            leaf_nodes.append(text_output)
            fundamental.append(check_fundamental(text_output))

In [29]:
# Check if equation is Tautology
def check_tautology(text_input):
    tautology = True

    # Reset lists
    global leaf_nodes
    global fundamental
    leaf_nodes = []
    fundamental = []

    RS_tree(text_input)
    for i in range(len(leaf_nodes)):
        print(leaf_nodes[i] + ' : ' + ('Fundamental' if fundamental[i] else 'Not Fundamental'))
        if fundamental[i] == False: tautology = False       # If a leaf is not fundamental, equation is not tautology
    print('Given formula is a Tautology' if tautology else 'Given formula is not a Tautology')

In [30]:
# To-do:
# Implement Console/UI for Input/Output + write documentation

In [31]:
# Test tautology
input = 'S1,A,→→A→BC→→AB→AC,S2'
check_tautology(input)

S1,A,A,A,~A,C,S2 : Fundamental
S1,A,A,~B,~A,C,S2 : Fundamental
S1,A,B,A,~A,C,S2 : Fundamental
S1,A,B,~B,~A,C,S2 : Fundamental
S1,A,~C,A,~A,C,S2 : Fundamental
S1,A,~C,~B,~A,C,S2 : Fundamental
Given formula is a Tautology


In [32]:
# Test tautology
input = '→→AB→~B~A'
check_tautology(input)

A,B,~A : Fundamental
~B,B,~A : Fundamental
Given formula is a Tautology


In [33]:
# Test tautology
input = '→→AB→→BC→AC'
check_tautology(input)

A,B,~A,C : Fundamental
A,~C,~A,C : Fundamental
~B,B,~A,C : Fundamental
~B,~C,~A,C : Fundamental
Given formula is a Tautology


In [34]:
# console import and setup

from cmd import Cmd
class SchemaPrompt(Cmd):
    intro = "Welcome! Enter a schema or type ? to view commands"
    def do_exit(self, inp):
        '''Exit the command shell'''
        print("Exiting")
        return True
    def do_example(self,inp):
        '''Display input and output of an example schema.'''
        print("Example Schema")
        print("Input: 'S1,A,→→A→BC→→AB→AC,S2'")
        print("Output: ")
        print(check_tautology("S1,A,→→A→BC→→AB→AC,S2"))
        pass
    def do_schema(self, inp):
        '''Check if given schema is a tautology'''
        if (inp == ""):
            print("Please enter a schema")
        else:
            print(check_tautology(inp))
        pass
        
SchemaPrompt().cmdloop()

Welcome! Enter a schema or type ? to view commands
(Cmd) ?

Documented commands (type help <topic>):
example  exit  help  schema

(Cmd) example
Example Schema
Input: 'S1,A,→→A→BC→→AB→AC,S2'
Output: 
S1,A,A,A,~A,C,S2 : Fundamental
S1,A,A,~B,~A,C,S2 : Fundamental
S1,A,B,A,~A,C,S2 : Fundamental
S1,A,B,~B,~A,C,S2 : Fundamental
S1,A,~C,A,~A,C,S2 : Fundamental
S1,A,~C,~B,~A,C,S2 : Fundamental
Given formula is a Tautology
None
(Cmd) schema('S1,A,→→A→BC→→AB→AC,S2')
('S1,A,A,A,~A,C,S2') : Fundamental
('S1,A,A,~B,~A,C,S2') : Fundamental
('S1,A,B,A,~A,C,S2') : Fundamental
('S1,A,B,~B,~A,C,S2') : Fundamental
('S1,A,~C,A,~A,C,S2') : Fundamental
('S1,A,~C,~B,~A,C,S2') : Fundamental
Given formula is a Tautology
None
(Cmd) exit
Exiting
